###### Imports and Settings

In [11]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
import sqlite3 as sq

In [12]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

# This notebook outlines the download and formatting process for the Zillow Home Value and Rental indices for counties and places in the GNRC operating region.  

Go to this page: https://www.zillow.com/research/data/  

+ Under "HOME VALUES", select Data Type "ZHVI All Homes (SFR, Condo/Co-op) Time Series, Smoothed, Seasonally Adjusted" and download this for Metro & US, State, County, and City. 
+ Under "RENTALS", select Data Type "ZORI (Smoothed, Seasonally Adjusted): All Homes Plus Multifamily Time Series" and download this for Metro & US, County, and City (State is not currently available).   

### Save these csvs as they come in the Data Downloads folder of Parent Data Gathering

In [13]:
hv_metrous = pd.read_csv('../../Data Downloads/Zillow_MetroUS_ZHVI.csv')
hv_state = pd.read_csv('../../Data Downloads/Zillow_State_ZHVI.csv')
hv_county = pd.read_csv('../../Data Downloads/Zillow_County_ZHVI.csv')
hv_place = pd.read_csv('../../Data Downloads/Zillow_City_ZHVI.csv')
rental_metrous = pd.read_csv('../../Data Downloads/Zillow_MetroUS_ZORI.csv')
rental_county = pd.read_csv('../../Data Downloads/Zillow_County_ZORI.csv')
rental_place = pd.read_csv('../../Data Downloads/Zillow_City_ZORI.csv')

The county codes are not FIPS codes, unsure what they are but our region contains the following:
Cheatham:2185, Davidson:2243, Dickson:1668 , Houston:1784, Humphreys:2728, Macon:623, Maury:632, Montgomery:2982, Robertson:2834, Rutherford:3016, Sumner:1407, Stewart:2044, Trousdale:2856, Williamson:3080, Wilson:1496, (KY) Allen:369 , (KY) Simpson:2028

In [14]:
#filter down the metro and us files to include only the US and the Nashville and Clarksville MSAs
metrous = [102001, 394902, 394471]
hv_metrous = hv_metrous.loc[hv_metrous['RegionID'].isin(metrous)].reset_index(drop = True)
rental_metrous = rental_metrous.loc[rental_metrous['RegionID'].isin(metrous)].reset_index(drop = True)
#filter the one state file
hv_state = hv_state.loc[hv_state['RegionID'] == 53].reset_index(drop = True)
#filter the county files, Simpson Co KY is RegionID 2028, but doesn't have associated data until recent years so not including for now
counties = [2185, 2243, 1668, 1784, 2728, 623, 632, 2982, 2834, 3016, 1407, 2044, 2856, 3080, 1496, 369]
hv_county = hv_county.loc[hv_county['RegionID'].isin(counties)].reset_index(drop = True)
rental_county = rental_county.loc[rental_county['RegionID'].isin(counties)].reset_index(drop = True)
#filter the place files
places = [41932, 30583, 10843, 30993, 49233, 45339, 11564, 32006, 46091, 25534, 42878, 39894, 19523, 6118, 26161, 54450, 7208, 27227, 29482, 41690]
hv_place = hv_place.loc[hv_place['RegionID'].isin(places)].reset_index(drop = True)
rental_place = rental_place.loc[rental_place['RegionID'].isin(places)].reset_index(drop = True)

## Fix up home value DFs

In [15]:
hv_place = hv_place.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName', 'State', 'Metro', 'CountyName']).set_index(('RegionName'))
hv_county = hv_county.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName', 'State', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS']).set_index(('RegionName'))
hv_state = hv_state.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName']).set_index(('RegionName'))
hv_metrous = hv_metrous.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName']).set_index(('RegionName'))

In [16]:
#append the home value dataframes
frame = [hv_place, hv_county, hv_state, hv_metrous]
emptyframe = pd.DataFrame()
for df in frame:
    emptyframe = emptyframe.append(df)    
hvs = emptyframe

C:\Users\jmccall\AppData\Local\Temp\ipykernel_15016\1944138516.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emptyframe = emptyframe.append(df)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_15016\1944138516.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emptyframe = emptyframe.append(df)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_15016\1944138516.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emptyframe = emptyframe.append(df)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_15016\1944138516.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emptyframe = emptyframe.append(df)


In [17]:
#transpose and rename the index to NAME
data = hvs.transpose().reset_index().rename(columns = {'index':'NAME'})

In [18]:
#adjust the geo names using str.split and index into the correct year
year = data['NAME'].str.split(pat = "-", expand = True)
data['Year'] = year[0]

In [19]:
#rename geos with module
data = data.rename(columns = tofullcensus)

In [20]:
#group by the average of the year, transpose, make the column a region name, then go from wide to long with .melt()
data = data.groupby(['Year']).mean()
data = data.transpose().reset_index()
data = data.rename(columns = {'RegionName':'NAME'})
data = data.set_index('NAME')
data = data.melt(value_name = 'Home Value', ignore_index = False)
data = data.reset_index()

C:\Users\jmccall\AppData\Local\Temp\ipykernel_15016\2975755765.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby(['Year']).mean()


In [21]:
#make sure year is formatted as an integer
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [22]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Home Value                                                                                                      \
Year                                   2000           2001           2002           2003      2004           2005           2006           2007   
NAME                                                                                                                                              
Allen County, Kentucky         64344.833333   66996.333333   68210.250000   69693.750000   72436.0   75754.666667   78024.750000   79704.833333   
Ashland City town, Tennessee  105943.416667  111582.833333  115210.166667  118399.333333  120450.5  125119.750000  131469.583333  139505.166667   

                                                                                                                                          \
Year                                   2008           2009      2010           2011           2012       2013        2014           2015   
NAME                                                                                                                                       
Allen County, Kentucky         79834.000000   79884.333333   79829.5   79282.333333   82387.000000   82293.25   84224.250   87459.416667   
Ashland City town, Tennessee  142778.083333  139541.500000  135594.0  131671.250000  130886.166667  132809.25  136858.875  146802.666667   

                                                                                                                                   
Year                                   2016       2017           2018           2019           2020           2021           2022  
NAME                                                                                                                               
Allen County, Kentucky         90266.333333   95505.25  100920.400000  105370.583333  108601.583333  121114.583333  137373.444444  
Ashland City town, Tennessee  153263.083333  174611.00  185306.333333  197662.250000  207455.500000  246753.666667  286882.555556

In [23]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Home Value                                                                                            \
                                       2000           2001           2002           2003           2004           2005           2006   
                                       None           None           None           None           None           None           None   
NAME                                                                                                                                    
Allen County, Kentucky         64344.833333   66996.333333   68210.250000   69693.750000   72436.000000   75754.666667   78024.750000   
Ashland City town, Tennessee  105943.416667  111582.833333  115210.166667  118399.333333  120450.500000  125119.750000  131469.583333   
Brentwood city, Tennessee     373468.000000  380417.000000  386755.250000  399329.166667  420839.416667  457219.250000  516847.166667   

                                                                                                                                                  \
                                       2007           2008           2009           2010           2011           2012       2013           2014   
                                       None           None           None           None           None           None       None           None   
NAME                                                                                                                                               
Allen County, Kentucky         79704.833333   79834.000000   79884.333333   79829.500000   79282.333333   82387.000000   82293.25   84224.250000   
Ashland City town, Tennessee  139505.166667  142778.083333  139541.500000  135594.000000  131671.250000  130886.166667  132809.25  136858.875000   
Brentwood city, Tennessee     563596.500000  551012.416667  520483.500000  501719.166667  505243.500000  512709.833333  540934.50  572712.166667   

                                                                                                                                                     
                                       2015           2016           2017           2018           2019           2020           2021          2022  
                                       None           None           None           None           None           None           None          None  
NAME                                                                                                                                                 
Allen County, Kentucky         87459.416667   90266.333333   95505.250000  100920.400000  105370.583333  108601.583333  121114.583333  1.373734e+05  
Ashland City town, Tennessee  146802.666667  153263.083333  174611.000000  185306.333333  197662.250000  207455.500000  246753.666667  2.868826e+05  
Brentwood city, Tennessee     602154.333333  634895.750000  663715.583333  688379.166667  713939.833333  746921.083333  929786.454545  1.220937e+06

In [24]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = df_pivot.columns.get_level_values(0).unique().tolist()
first_level

['Home Value']

In [25]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [26]:
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [27]:
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'Zillow'

In [28]:
#final check
data.head()

,NAME,Year,Time Frame,Home Value,Home Value % Change,Home Value Change,GEO_ID,Source
0,"Allen County, Kentucky",2000,None,64344.833333,NaN,NaN,0500000US21003,Zillow
1,"Allen County, Kentucky",2001,None,66996.333333,NaN,NaN,0500000US21003,Zillow
2,"Allen County, Kentucky",2002,None,68210.250000,NaN,NaN,0500000US21003,Zillow
3,"Allen County, Kentucky",2003,None,69693.750000,NaN,NaN,0500000US21003,Zillow
4,"Allen County, Kentucky",2004,None,72436.000000,NaN,NaN,0500000US21003,Zillow


In [29]:
data['NAME'].unique()

array(['Allen County, Kentucky', 'Ashland City town, Tennessee',
       'Brentwood city, Tennessee', 'Cheatham County, Tennessee',
       'Clarksville city, Tennessee', 'Columbia city, Tennessee',
       'Davidson County, Tennessee', 'Dickson County, Tennessee',
       'Fairview city, Tennessee', 'Franklin city, Kentucky',
       'Gallatin city, Tennessee', 'Hendersonville city, Tennessee',
       'Houston County, Tennessee', 'Humphreys County, Tennessee',
       'La Vergne city, Tennessee', 'Lebanon city, Tennessee',
       'Macon County, Tennessee', 'Maury County, Tennessee',
       'Millersville city, Tennessee', 'Montgomery County, Tennessee',
       'Mount Juliet city, Tennessee', 'Murfreesboro city, Tennessee',
       'Nashville-Davidson metropolitan government (balance), Tennessee',
       'Nolensville town, Tennessee', 'Robertson County, Tennessee',
       'Rutherford County, Tennessee', 'Smyrna town, Tennessee',
       'Spring Hill city, Tennessee', 'Springfield city, Tennesse

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10488 entries, 0 to 10487
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NAME                 10488 non-null  object 
 1   Year                 10488 non-null  object 
 2   Time Frame           10488 non-null  object 
 3   Home Value           874 non-null    float64
 4   Home Value % Change  9614 non-null   float64
 5   Home Value Change    9614 non-null   float64
 6   GEO_ID               9936 non-null   object 
 7   Source               10488 non-null  object 
dtypes: float64(3), object(5)
memory usage: 655.6+ KB


In [31]:
#export to the SQLite database as Zillow annual data
conn = sq.connect('../../Outputs/Dem_Transpo_Housing_Collection.db')
data.to_sql('Zillow_HomeValue_Annual_Change', conn, if_exists = 'replace', index = False)

10488

## Fix Up Rental DFs

In [32]:
rental_place = rental_place.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName', 'State', 'Metro', 'CountyName']).set_index(('RegionName'))
rental_county= rental_county.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName', 'State', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS']).set_index(('RegionName'))
rental_metrous = rental_metrous.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName']).set_index(('RegionName'))

In [33]:
#append the home value dataframes
frame = [rental_place, rental_county, rental_metrous]
emptyframe = pd.DataFrame()
for df in frame:
    emptyframe = emptyframe.append(df)  
rental = emptyframe

C:\Users\jmccall\AppData\Local\Temp\ipykernel_15016\3274071810.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emptyframe = emptyframe.append(df)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_15016\3274071810.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emptyframe = emptyframe.append(df)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_15016\3274071810.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emptyframe = emptyframe.append(df)


In [34]:
#transpose and rename the index to NAME
data = rental.transpose().reset_index().rename(columns = {'index':'NAME'})

In [35]:
#adjust the geo names using str.split and index into the correct year
year = data['NAME'].str.split(pat = "-", expand = True)
data['Year'] = year[0]

In [36]:
#rename using module
data = data.rename(columns = tofullcensus)

In [37]:
#group by the average of the year, transpose, make the column a region name, then go from wide to long with .melt()
data = data.groupby(['Year']).mean()
data = data.transpose().reset_index()
data = data.rename(columns = {'RegionName':'NAME'})
data = data.set_index('NAME')
data = data.melt(value_name = 'Average Rent', ignore_index = False)
data = data.reset_index()

C:\Users\jmccall\AppData\Local\Temp\ipykernel_15016\1759768606.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby(['Year']).mean()


In [38]:
#make sure year is formatted as an integer
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [39]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Average Rent                                                                                           
Year                                 2015         2016         2017         2018         2019         2020         2021         2022
NAME                                                                                                                                
Ashland City town, Tennessee   175.277818   271.722477   414.051957   794.348836   977.301430  1250.117199  1314.676772  1499.093735
Brentwood city, Tennessee     2027.976502  2083.788301  2168.446406  2214.974303  2270.834849  2328.509989  2679.600785  3179.383195

In [40]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Average Rent                                                                                           
                                     2015         2016         2017         2018         2019         2020         2021         2022
                                     None         None         None         None         None         None         None         None
NAME                                                                                                                                
Ashland City town, Tennessee   175.277818   271.722477   414.051957   794.348836   977.301430  1250.117199  1314.676772  1499.093735
Brentwood city, Tennessee     2027.976502  2083.788301  2168.446406  2214.974303  2270.834849  2328.509989  2679.600785  3179.383195
Cheatham County, Tennessee     448.336789   511.597978   678.261496   925.004323  1079.649920  1224.532422  1324.128357  1551.356073

In [41]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = df_pivot.columns.get_level_values(0).unique().tolist()
first_level

['Average Rent']

In [42]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [43]:
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [44]:
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'Zillow'

In [45]:
#final check
data.head()

,NAME,Year,Time Frame,Average Rent,Average Rent % Change,Average Rent Change,GEO_ID,Source
0,"Ashland City town, Tennessee",2015,None,175.277818,NaN,NaN,1600000US4702180,Zillow
1,"Ashland City town, Tennessee",2016,None,271.722477,NaN,NaN,1600000US4702180,Zillow
2,"Ashland City town, Tennessee",2017,None,414.051957,NaN,NaN,1600000US4702180,Zillow
3,"Ashland City town, Tennessee",2018,None,794.348836,NaN,NaN,1600000US4702180,Zillow
4,"Ashland City town, Tennessee",2019,None,977.301430,NaN,NaN,1600000US4702180,Zillow


In [46]:
data['NAME'].unique()

array(['Ashland City town, Tennessee', 'Brentwood city, Tennessee',
       'Cheatham County, Tennessee', 'Clarksville city, Tennessee',
       'Columbia city, Tennessee', 'Davidson County, Tennessee',
       'Fairview city, Tennessee', 'Franklin city, Kentucky',
       'Gallatin city, Tennessee', 'Hendersonville city, Tennessee',
       'La Vergne city, Tennessee', 'Lebanon city, Tennessee',
       'Maury County, Tennessee', 'Millersville city, Tennessee',
       'Montgomery County, Tennessee', 'Mount Juliet city, Tennessee',
       'Murfreesboro city, Tennessee',
       'Nashville-Davidson metropolitan government (balance), Tennessee',
       'Nolensville town, Tennessee', 'Robertson County, Tennessee',
       'Rutherford County, Tennessee', 'Smyrna town, Tennessee',
       'Spring Hill city, Tennessee', 'Springfield city, Tennessee',
       'Sumner County, Tennessee', 'Thompsons Station', 'United States',
       'White House city, Tennessee', 'Williamson County, Tennessee',
       'W

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   1080 non-null   object 
 1   Year                   1080 non-null   object 
 2   Time Frame             1080 non-null   object 
 3   Average Rent           240 non-null    float64
 4   Average Rent % Change  840 non-null    float64
 5   Average Rent Change    840 non-null    float64
 6   GEO_ID                 1008 non-null   object 
 7   Source                 1080 non-null   object 
dtypes: float64(3), object(5)
memory usage: 67.6+ KB


In [38]:
#export to the SQLite database as Zillow annual data
conn = sq.connect('../../Outputs/Dem_Transpo_Housing_Collection.db')
data.to_sql('Zillow_Rent_Annual_Change', conn, if_exists = 'replace', index = False)

1080